In [3]:
import pandas as pd
import numpy as np
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

In [13]:
# !pip install pyarrow
# !pip install openpyxl
!pip install azure.datalake.store

  Using cached azure_datalake_store-0.0.48-py2.py3-none-any.whl (53 kB)
  Using cached adal-1.2.3-py2.py3-none-any.whl (53 kB)


# Normalizando base de vendas pelos 12 meses anteriores

In [4]:
def normalizando_volume(df, n_dias=365):
    '''
    Normalizando os volumes pelos n_dias anteriores (default: 365 -> 12 meses)
    '''
    
    #Agrupando os volumes dos terminais por dia
    df_vendas_agg = df.sort_values(by=['Date']).set_index(['Date'], drop=True) \
                                               .groupby(by=['UF', 'Produto', 'Segmento']).resample('D').sum()
    
    #Excluindo Volumes negativos
    df_vendas_agg = df_vendas_agg.query("Volume >= 0")
    
    #Copiando DataFrame para posterior merge
    df_vendas_agg_merge = df_vendas_agg.copy()
    
    #Resetando index
    df_vendas_agg.reset_index(inplace=True)
    df_vendas_agg.set_index(['Date'], drop=True, inplace=True)
    
    #Calculo da média dos n_dias anteriores
    vol_medio = df_vendas_agg.groupby(by=['UF', 'Produto', 'Segmento']).rolling(n_dias).mean()
    vol_medio.rename(columns={'Volume': 'vol_medio'}, inplace=True)
    
    #Merge nas bases para normalização
    vendas_dia_final = df_vendas_agg_merge.merge(vol_medio, left_index=True, right_index=True)
    
    #Resetando index
    vendas_dia_final.reset_index(inplace=True)
    
    #Normalizando pela média dos n_dias anteriores
    vendas_dia_final['vol_norm'] = np.where(vendas_dia_final.vol_medio != 0.0,
                                            vendas_dia_final.Volume / vendas_dia_final.vol_medio,
                                            0.0)
    
    #Filtrando os produtos
    vendas_dia_final = vendas_dia_final.query("Date >= '2012-04-01' and (Produto != 'Etanol' or Segmento != 'C') and \
                                                                        (Produto != 'Gasolina' or Segmento != 'C')")
    
    return vendas_dia_final.drop(['vol_medio'], axis=1)

In [5]:
def calculando_pre_pos_feriado(df_dias_uteis):
    '''
    Calculando se os dias são feriados, pré feriado ou pós feriado
    '''

    #Criando colunas dia_util e feriado
    df_dias_uteis['dia_util'] = np.where((df_dias_uteis.Nac == '0') & (df_dias_uteis.Est == '0') & (df_dias_uteis.Data.dt.weekday < 6), 1, 0)
    df_dias_uteis['feriado'] = np.where((df_dias_uteis.Nac == '1') | (df_dias_uteis.Est == '1'), 1, 0)
    
    #Agrupando por Dia e UF para descobrir dias pós e pré feriado
    calendario_uf = df_dias_uteis.groupby(['UF', 'Data']).agg({'feriado': sum})
    calendario_uf['feriado'] = np.where(calendario_uf.feriado >= 1, 1, 0)
    
    #Shift para analisar se o dia posterior ou anterior foram feriados
    calendario_uf['pos_feriado'] = calendario_uf.feriado.shift(1)
    calendario_uf['pre_feriado'] = calendario_uf.feriado.shift(-1)
    calendario_uf.reset_index(inplace=True)
    
    #Inserindo colunas de pré e pós feriado no DataFrame original
    df_dias_uteis = df_dias_uteis.merge(calendario_uf, on=['UF', 'Data', 'feriado'])
    dias_uteis_final = df_dias_uteis[['UF', 'Data', 'dia_util', 'feriado', 'pos_feriado', 'pre_feriado']]
    
    #Ajustando a base
    dias_uteis_final.fillna(1, inplace=True) #2019-12-31 é pré-feriado e estava como NaN
    dias_uteis_final.pos_feriado = dias_uteis_final.pos_feriado.astype(int)
    dias_uteis_final.pre_feriado = dias_uteis_final.pre_feriado.astype(int)
    
    return dias_uteis_final

In [6]:
def calculando_tipo_dia(df_dias_uteis, date_max):
    '''
    Calculando tipos de dia
        - Dia Util
        - Feriado
        - Pré e pós feriado
        - Dezembro
    '''
    
    #Convertendo data_max em datetime
    date_max = pd.to_datetime(date_max, format='%Y-%m-%d')
    
    #Filtrando para o período definido
    df_dias_uteis = df_dias_uteis[(df_dias_uteis.Data <= date_max)]
    
    #Inserindo dia util, feriado, pré e pós feriado na base
    dias_uteis_final = calculando_pre_pos_feriado(df_dias_uteis)
    
    # # # Definindo tipos de dias
    dias_uteis_final['dia_tipo_semana'] = 0

    # # Jan - Nov

    #Semana normal
    dias_uteis_final['dia_tipo_semana'] = np.where((dias_uteis_final.Data.dt.weekday == 0) & (dias_uteis_final.dia_util == 1), 'segunda', dias_uteis_final.dia_tipo_semana)
    dias_uteis_final['dia_tipo_semana'] = np.where((dias_uteis_final.Data.dt.weekday == 1) & (dias_uteis_final.dia_util == 1), 'terca', dias_uteis_final.dia_tipo_semana)
    dias_uteis_final['dia_tipo_semana'] = np.where((dias_uteis_final.Data.dt.weekday == 2) & (dias_uteis_final.dia_util == 1), 'quarta', dias_uteis_final.dia_tipo_semana)
    dias_uteis_final['dia_tipo_semana'] = np.where((dias_uteis_final.Data.dt.weekday == 3) & (dias_uteis_final.dia_util == 1), 'quinta', dias_uteis_final.dia_tipo_semana)
    dias_uteis_final['dia_tipo_semana'] = np.where((dias_uteis_final.Data.dt.weekday == 4) & (dias_uteis_final.dia_util == 1), 'sexta', dias_uteis_final.dia_tipo_semana)
    dias_uteis_final['dia_tipo_semana'] = np.where((dias_uteis_final.Data.dt.weekday == 5) & (dias_uteis_final.dia_util == 1), 'sabado', dias_uteis_final.dia_tipo_semana)
    dias_uteis_final['dia_tipo_semana'] = np.where((dias_uteis_final.Data.dt.weekday == 6) & (dias_uteis_final.dia_util == 0), 'domingo', dias_uteis_final.dia_tipo_semana)

    #Feriado dia de semana
    dias_uteis_final['dia_tipo_semana'] = np.where((dias_uteis_final.Data.dt.weekday == 0) & (dias_uteis_final.feriado == 1), 'segunda_feriado', dias_uteis_final.dia_tipo_semana)
    dias_uteis_final['dia_tipo_semana'] = np.where((dias_uteis_final.Data.dt.weekday == 1) & (dias_uteis_final.feriado == 1), 'terca_feriado', dias_uteis_final.dia_tipo_semana)
    dias_uteis_final['dia_tipo_semana'] = np.where((dias_uteis_final.Data.dt.weekday == 2) & (dias_uteis_final.feriado == 1), 'quarta_feriado', dias_uteis_final.dia_tipo_semana)
    dias_uteis_final['dia_tipo_semana'] = np.where((dias_uteis_final.Data.dt.weekday == 3) & (dias_uteis_final.feriado == 1), 'quinta_feriado', dias_uteis_final.dia_tipo_semana)
    dias_uteis_final['dia_tipo_semana'] = np.where((dias_uteis_final.Data.dt.weekday == 4) & (dias_uteis_final.feriado == 1), 'sexta_feriado', dias_uteis_final.dia_tipo_semana)
    dias_uteis_final['dia_tipo_semana'] = np.where((dias_uteis_final.Data.dt.weekday == 5) & (dias_uteis_final.feriado == 1), 'sabado_feriado', dias_uteis_final.dia_tipo_semana)
    dias_uteis_final['dia_tipo_semana'] = np.where((dias_uteis_final.Data.dt.weekday == 6) & (dias_uteis_final.feriado == 1), 'domingo_feriado', dias_uteis_final.dia_tipo_semana)

    #Dia pré feriado
    dias_uteis_final['dia_tipo_semana'] = np.where((dias_uteis_final.Data.dt.weekday == 0) & (dias_uteis_final.pre_feriado == 1), 'segunda_pre_feriado', dias_uteis_final.dia_tipo_semana)
    dias_uteis_final['dia_tipo_semana'] = np.where((dias_uteis_final.Data.dt.weekday == 1) & (dias_uteis_final.pre_feriado == 1), 'terca_pre_feriado', dias_uteis_final.dia_tipo_semana)
    dias_uteis_final['dia_tipo_semana'] = np.where((dias_uteis_final.Data.dt.weekday == 2) & (dias_uteis_final.pre_feriado == 1), 'quarta_pre_feriado', dias_uteis_final.dia_tipo_semana)
    dias_uteis_final['dia_tipo_semana'] = np.where((dias_uteis_final.Data.dt.weekday == 3) & (dias_uteis_final.pre_feriado == 1), 'quinta_pre_feriado', dias_uteis_final.dia_tipo_semana)
    dias_uteis_final['dia_tipo_semana'] = np.where((dias_uteis_final.Data.dt.weekday == 4) & (dias_uteis_final.pre_feriado == 1), 'sexta_pre_feriado', dias_uteis_final.dia_tipo_semana)
    dias_uteis_final['dia_tipo_semana'] = np.where((dias_uteis_final.Data.dt.weekday == 5) & (dias_uteis_final.pre_feriado == 1), 'sabado_pre_feriado', dias_uteis_final.dia_tipo_semana)
    dias_uteis_final['dia_tipo_semana'] = np.where((dias_uteis_final.Data.dt.weekday == 6) & (dias_uteis_final.pre_feriado == 1), 'domingo_pre_feriado', dias_uteis_final.dia_tipo_semana)

    #Dia pós feriado
    dias_uteis_final['dia_tipo_semana'] = np.where((dias_uteis_final.Data.dt.weekday == 0) & (dias_uteis_final.pos_feriado == 1), 'segunda_pos_feriado', dias_uteis_final.dia_tipo_semana)
    dias_uteis_final['dia_tipo_semana'] = np.where((dias_uteis_final.Data.dt.weekday == 1) & (dias_uteis_final.pos_feriado == 1), 'terca_pos_feriado', dias_uteis_final.dia_tipo_semana)
    dias_uteis_final['dia_tipo_semana'] = np.where((dias_uteis_final.Data.dt.weekday == 2) & (dias_uteis_final.pos_feriado == 1), 'quarta_pos_feriado', dias_uteis_final.dia_tipo_semana)
    dias_uteis_final['dia_tipo_semana'] = np.where((dias_uteis_final.Data.dt.weekday == 3) & (dias_uteis_final.pos_feriado == 1), 'quinta_pos_feriado', dias_uteis_final.dia_tipo_semana)
    dias_uteis_final['dia_tipo_semana'] = np.where((dias_uteis_final.Data.dt.weekday == 4) & (dias_uteis_final.pos_feriado == 1), 'sexta_pos_feriado', dias_uteis_final.dia_tipo_semana)
    dias_uteis_final['dia_tipo_semana'] = np.where((dias_uteis_final.Data.dt.weekday == 5) & (dias_uteis_final.pos_feriado == 1), 'sabado_pos_feriado', dias_uteis_final.dia_tipo_semana)
    dias_uteis_final['dia_tipo_semana'] = np.where((dias_uteis_final.Data.dt.weekday == 6) & (dias_uteis_final.pos_feriado == 1), 'domingo_pos_feriado', dias_uteis_final.dia_tipo_semana)

    # # Dez

    dias_uteis_final['dia_tipo_semana'] = np.where(dias_uteis_final.Data.dt.month == 12, dias_uteis_final.dia_tipo_semana + '_dez', dias_uteis_final.dia_tipo_semana)
    
    return dias_uteis_final

In [7]:
def create_master_table(vendas_dia_norm, dias_uteis):
    '''
    Criação da master table
        - Base dias úteis
        - Base volume normalizado
    '''
    
    #Merge nas bases
    df_all = vendas_dia_norm.merge(dias_uteis, left_on=['Date', 'UF'], right_on=['Data', 'UF'], how='inner')
    
    #Dropando colunas que não interessam
    df_all.drop(['Data', 'dia_util', 'feriado', 'pos_feriado', 'pre_feriado'], axis=1, inplace=True)
    
    return df_all

In [8]:
def calculo_vol_medio_dias_uteis(df):
    '''
    Calcula volume médio para dias úteis
    '''
    
    #Criando lista de dias úteis
    dia_util = ['segunda', 'terca', 'quarta', 'quinta', 'sexta', 'segunda_dez', 
                'terca_dez', 'quarta_dez', 'quinta_dez', 'sexta_dez']
    
    #Adicionando coluna dia_util
    df['dia_util'] = np.where(df.dia_tipo_semana.isin(dia_util), 1, 0)

    #Calculando Volume medio para dia util de cada UF/Produto/Segmento
    df_dia_util = df.copy()
    df_dia_util = df_dia_util.groupby(['UF', 'Produto', 'Segmento', 'dia_util']).mean().reset_index()
    
    #Filtrando apenas para dias úteis
    df_dia_util = df_dia_util.query("dia_util == 1")[['UF', 'Produto', 'Segmento', 'dia_util', 'vol_norm']]
    
    #Renomeando a coluna com o volume medio dos dias úteis
    df_dia_util.rename(columns={'vol_norm': 'vol_medio_dia_util'}, inplace=True)
    
    return df_dia_util.drop(['dia_util'], axis=1)

In [9]:
def validacao_pesos(df):
    '''
    Cálculo do erro de representação do dia frente a semana
    '''
    
    #Criando colunas com o número da semana e ano
    df['semana_num'] = df.Date.dt.weekofyear
    df['ano'] = df.Date.dt.year
    
    #Soma semanal UF/Produto/Segmento
    df_week = df.set_index(['Date']).groupby(['UF', 'Produto', 'Segmento']).resample('W').sum().reset_index()

    #Criando coluna com numero da semana e ano para posterior Merge
    df_week['semana_num'] = df_week.Date.dt.weekofyear
    df_week['ano'] = df_week.Date.dt.year
    
    #Tratando o DataFrame
    df_week.drop(['Date', 'vol_norm'], axis=1, inplace=True)
    df_week.rename(columns={'Volume': 'vol_semanal', 'peso': 'peso_semanal'}, inplace=True)
    
    #Merge com a base completa
    df = df.merge(df_week, on=['UF', 'Produto', 'Segmento', 'semana_num', 'ano'])
    df.drop(['vol_norm'], axis=1, inplace=True)
    
    #Validação
    df["Representatividade_Peso_Dia_na_Semana"] = np.where(df.peso_semanal > 0,
                                                           df.peso / df.peso_semanal,
                                                           df.peso)

    df["Representatividade_Volume_Dia_na_Semana"] = np.where(df.vol_semanal > 0,
                                                             df.Volume / df.vol_semanal,
                                                             df.Volume)

    df["Erro_Representatividade"] = np.where(df.Representatividade_Volume_Dia_na_Semana > 0,
                                             abs((df.Representatividade_Peso_Dia_na_Semana - df.Representatividade_Volume_Dia_na_Semana) / df.Representatividade_Volume_Dia_na_Semana),
                                             df.Representatividade_Peso_Dia_na_Semana)

    df["Erro_Representatividade"] = np.where(df.Erro_Representatividade > 1,
                                             1.0,
                                             df.Erro_Representatividade)
    
    #Criando coluna Erro * Volume para realização da média ponderada
    df['Erro_Ponderado'] = (df.Erro_Representatividade * df.Volume)

    #Calculando erro ponderado
    err_ponderado = df.Erro_Ponderado.sum() / df.Volume.sum()

    #Calculando erro medio
    err_medio = df.Erro_Representatividade.mean()

    print("Erro medio: ",err_medio)
    print("Erro ponderado: ",err_ponderado)
    
    return df[['UF', 'Produto', 'Segmento', 'Date', 'Volume', 'dia_tipo_semana', 'peso', 'Erro_Representatividade']]

In [10]:
from itertools import product

def reply_future(df, df_dias_uteis):
    '''
    Função para replicação dos pesos encontrados para as datas futuras
    '''
    
    #Selecionando o período futuro no DataFrame df_dias_uteis
    df_dias_uteis_future = df_dias_uteis[df_dias_uteis.Data >= df.Date.max()]
    
    #Criando pool para inserir UF, Produto e Segmento para cada dia da base df_dias_uteis_future
    lista_uf = list(df_validado.UF.unique())
    lista_prod = list(df_validado.Produto.unique())
    lista_segm = list(df_validado.Segmento.unique())
    pool = product(lista_uf, lista_prod, lista_segm) 
    
    #Criando df vazio com as colunas de interesse
    df_uf_prod_segm = pd.DataFrame(columns=['UF', 'Produto', 'Segmento'])

    #Adicionando UF, Produto e Segmento ao df criado anteriormente
    for uf, prod, segm in pool:
        df_uf_prod_segm = df_uf_prod_segm.append({'UF': uf, 'Produto': prod, 'Segmento': segm}, ignore_index=True)

    #Filtrando dados
    df_uf_prod_segm = df_uf_prod_segm.query("(Produto != 'Etanol' or Segmento != 'C') and \
                                             (Produto != 'Gasolina' or Segmento != 'C')")
    
    #Merge para criar DataFrame com datas futuras para cada UF, Produto e Segmento
    df_future = df_dias_uteis_future.merge(df_uf_prod_segm, on=['UF'])
    df_future.drop(['dia_util', 'feriado', 'pos_feriado', 'pre_feriado'], axis=1, inplace=True)
    
    #Encontrando o Erro de Representatividade médio para cada granularidade
    df_err_medio = df.groupby(['UF', 'Produto', 'Segmento', 'dia_tipo_semana']) \
                     .agg({'Erro_Representatividade': np.mean}) \
                     .reset_index()
    
    #Dropando colunas que não interessam 
    df.drop(['Date', 'Volume', 'Erro_Representatividade'], axis=1, inplace=True)
    
    #Merge final
    df_future_final = df.merge(df_err_medio, on=['UF', 'Produto', 'Segmento', 'dia_tipo_semana']).drop_duplicates()
    df_future_final = df_future_final.merge(df_future, on=['UF', 'Produto', 'Segmento', 'dia_tipo_semana'])
    
    #Arrumando o DataFrame
    df_future_final = df_future_final[['UF', 'Produto', 'Segmento', 'Data', 'dia_tipo_semana', 'peso', 'Erro_Representatividade']]
    df_future_final.rename(columns={'Data': 'Date'}, inplace=True)
    
    return df_future_final

In [20]:
#Abrindo base
# df_vendas = pd.read_parquet('bases/vendas_dia.parquet')
#df_vendas = pd.read_parquet('vendas_dia.parquet', engine='pyarrow')

#Limpando a base
a.toPandas()
a = a.drop_duplicates()
a = a.dropna()

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:45037)
Traceback (most recent call last):
  File "/opt/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1159, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 985, in send_command
    response = connection.send_command(command)
  File "/opt/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1164, in send_command
    "Error while receiving", e, proto.ERROR_ON_RECEIVE)
py4j.protocol.Py4JNetworkError: Error while receiving

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py"

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:45037)
Traceback (most recent call last):
  File "/opt/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1159, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 985, in send_command
    response = connection.send_command(command)
  File "/opt/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1164, in send_command
    "Error while receiving", e, proto.ERROR_ON_RECEIVE)
py4j.protocol.Py4JNetworkError: Error while receiving

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py"

Py4JNetworkError: An error occurred while trying to connect to the Java server (127.0.0.1:45037)

In [10]:
#Normalizando os volumes
vendas_dia_norm = normalizando_volume(df_vendas)

In [11]:
#Lendo a base dias uteis e filtrando para UF
#     df_dias_uteis = pd.read_parquet('bases/dias_uteis_chave.parquet')
df_dias_uteis = pd.read_parquet('dias_uteis_chave.parquet')
df_dias_uteis = df_dias_uteis[['UF', 'Data', 'Nac', 'Est']].drop_duplicates()

In [12]:
#Calculando tipos de dias
dias_uteis = calculando_tipo_dia(df_dias_uteis, date_max='2021-03-31')

In [13]:
#Master table
master = create_master_table(vendas_dia_norm, dias_uteis)

In [14]:
#Calculando o volume médio para cada tipo de dia
master_vol_tipo_dia = master.groupby(['UF', 'Produto', 'Segmento', 'dia_tipo_semana']) \
                            .agg({'vol_norm': np.mean}) \
                            .reset_index()

In [16]:
#Calculando o volume médio de dias úteis
master_vol_tipo_dia_aux = master_vol_tipo_dia.copy()
master_vol_dias_uteis = calculo_vol_medio_dias_uteis(master_vol_tipo_dia_aux)

In [18]:
#Merge nas bases e cálculo do peso
master_pesos = master_vol_tipo_dia.merge(master_vol_dias_uteis, on=['UF', 'Produto', 'Segmento'])

master_pesos['peso'] = np.where(~master_pesos.vol_norm.isnull(), master_pesos.vol_norm / master_pesos.vol_medio_dia_util, 0)
master_pesos.drop(['vol_norm', 'vol_medio_dia_util'], axis=1, inplace=True)

In [19]:
#Merge com a master table original
master = master.merge(master_pesos, on=['UF', 'Produto', 'Segmento', 'dia_tipo_semana'])

In [106]:
#Validando os pesos encontrados
df_validado = validacao_pesos(master)

Erro medio:  0.23286281713013263
Erro ponderado:  0.1389134123765066


In [109]:
#Replicando os pesos para as datas futuras
df_presente_final = df_validado.copy()
df_future_final = reply_future(df_validado, dias_uteis)

#Filtrando colunas de interesse
df_presente_final.drop(['Volume'], axis=1, inplace=True)

In [112]:
#Concatenando os DataFrames com passado e futuro
df_final = pd.concat([df_presente_final, df_future_final])

In [119]:
#Filtrando a base
df_final = df_final.drop(['dia_tipo_semana'], axis=1).sort_values(by=['UF', 'Date'])
df_final.rename(columns={'peso': 'Peso'}, inplace=True)

In [124]:
#Exportando para Excel
df_final.to_excel('racional_dias_uteis_peso.xlsx', index=False)

#Exportando para parquet
df_final.to_parquet('racional_dias_uteis_peso.parquet', index=False, engine='pyarrow')

In [14]:
from azure.datalake.store import core, lib, multithread

In [15]:
from pyspark.sql import SparkSession

In [16]:
spark = SparkSession.builder \
            .master("local[8]") \
            .appName("airflow_app") \
            .config('spark.executor.memory', '16g') \
            .config('spark.driver.memory', '16g') \
            .config('spark.sql.execution.pandas.respectSessionTimeZone', False) \
            .config("spark.driver.maxResultSize", "2048MB") \
            .config("spark.port.maxRetries", "100") \
            .config("spark.sql.execution.arrow.enabled", "true") \
            .getOrCreate()

In [17]:
adlCreds = lib.auth(url_suffix='raizenprd01', resource='https://datalake.azure.net/')

To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code BSN6BZG8Z to authenticate.


In [18]:
spark.conf.set("fs.adl.oauth2.access.token.provider.type", "RefreshToken")
spark.conf.set("fs.adl.oauth2.client.id", adlCreds.token['client'])
spark.conf.set("fs.adl.oauth2.refresh.token", adlCreds.token['refreshToken'])

In [19]:
a = spark.read.parquet('adl://raizenprd01.azuredatalakestore.net/ldt_dev/sandbox/previsao_demanda/dieffgg/06_mastertable/master')